In [7]:
from transformers import ElectraModel, ElectraTokenizer
from transformers import ElectraForSequenceClassification
from transformers import AutoTokenizer, AutoModelForSemanticSegmentation, TrainingArguments, Trainer

import torch

import pandas as pd
from konlpy.tag import Mecab
import re
from tqdm import tqdm, tqdm_notebook

from sklearn.metrics import precision_recall_fscore_support, accuracy_score, classification_report

# 1차 모델

In [3]:
# labeling 된 데이터 불러오기
survey = pd.read_csv("./_data/survey.csv", index_col=0)
survey.fillna(0, inplace=True)
survey = survey.reset_index()

# 데이터 중에서 작업을 하고자 하는 라벨만 가져오기
guide_data = survey[['content_id', 'guide']]

# content_id 중에서 댓글이 같이 추가된 데이터 정리
guide_data['content_id'] = guide_data['content_id'].apply(lambda x: x.split(',')[0] if len(str(x)) > 6 else x)

# content_id 가 0인 데이터 제외 -> content_id 모두 int type 으로 변경 (추후 Merge 를 위함)
guide_data = guide_data[guide_data['content_id'] != 0].reset_index().drop(columns=['index'])
guide_data['content_id'] = guide_data['content_id'].astype(int)

# 전체 review data 불러오기
review_all = pd.read_csv("./_data/reviews.csv", index_col=0)

# 전체 review data 중에서 survey data에 있는 댓글만 가져오기
survey_content = review_all.loc[review_all['id'].isin(guide_data['content_id'])][['id', 'content']]
# merge 를 위해서 id 컬럼명 통일하기
survey_content.columns=['content_id', 'content']

# 통일된 content_id 를 기반으로 데이터 merge
guide_data = pd.merge(guide_data, survey_content)

# 추후 원활한 계산을 위해서 숫자 부분은 모두 int 로 바꿔줌
guide_data['guide'] = guide_data['guide'].astype(int)

# 최소한의 전처리
def cleaned_content(text):
    d = re.sub('\n', '. ', text) # 줄바꿈 > .
    d = re.sub('[^가-힣0-9a-zA-Z ]{2,}', ".", d) # 특수문자 두개 이상인거 .으로 변경
    return d

guide_data['content'] = guide_data['content'].apply(cleaned_content)

final_df = guide_data[['content', 'guide']]

# 라벨별 개수 확인
print(final_df['guide'].value_counts())
print('\n')
train_data = final_df.sample(frac=0.8, random_state=42).reset_index().drop(columns='index')
print('train_data', train_data['guide'].value_counts())
test_data = final_df.drop(train_data.index).reset_index().drop(columns='index')
print('\n', 'test_data', test_data['guide'].value_counts())
print('\n')
# 중복 데이터 제거(데이터 분리 후 중복이 생길 수 있어서 데이터 분리 후 중복 데이터 처리 진행)

# 데이터셋 개수 확인
print('중복 제거 전 학습 데이터셋: {}'.format(len(train_data)))
print('중복 제거 전 테스트 데이터셋: {}'.format(len(test_data)))
print('\n')
# 중복 데이터 제거
train_data.drop_duplicates(subset=['content'], inplace=True)
test_data.drop_duplicates(subset=['content'], inplace=True)
print('\n')
# 데이터셋 개수 확인
print('중복 제거 후 학습 데이터셋: {}'.format(len(train_data)))
print('중복 제거 후 테스트 데이터셋: {}'.format(len(test_data)))

/var/folders/t4/314qsssd1p718wry9j0bh9000000gn/T/ipykernel_66259/3970883154.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  guide_data['content_id'] = guide_data['content_id'].apply(lambda x: x.split(',')[0] if len(str(x)) > 6 else x)


guide
 0    4774
-1     200
 1      49
Name: count, dtype: int64


train_data guide
 0    3820
-1     162
 1      36
Name: count, dtype: int64

 test_data guide
 0    965
-1     35
 1      5
Name: count, dtype: int64


중복 제거 전 학습 데이터셋: 4018
중복 제거 전 테스트 데이터셋: 1005




중복 제거 후 학습 데이터셋: 4003
중복 제거 후 테스트 데이터셋: 1001


In [ ]:
# 토큰에 추가할 단어 -> '방탈출'이라는 도메인 지시기에 근거한 용어, 분리되어서는 안 되기 때문에 별도로 추가 작업 진행
addword = ['공테', '약공테', '감테', '창공', '갑툭튀', '삐딱', '삑', '꽝', '삑딱쾅', '삑딱쾅', '삑딱', '쫄', '극쫄',
            '극극쫄', '쫄팟', '쫄탱', '쫄보', '극', '약탱', '탱쫄', '극극극', '뉴비', '하드캐리', '극혐', '피지컬',
            '어거지', '뚝배기', '뚝문', '셀뚝', '억까', '트롤링', '트롤짓', '흙길', '풀길', '꽃길', '풀꽃', '꽃다발',
            '꽃밭', '웰메이드', '인생테마', '머글', '방린이', '방유아', '방세포', '방태아', '방탈러', '과몰입러', '옵저버',
            '리트', '연방', '혼방', '혼불', '워킹', '워크인', '장치방', '문제방', '직렬', '병렬', '육각형', '볼드', '볼드충', '에바',
            '가이드', '조도', '조명', '밝기', '어두움', '인테리어', '비주얼', '소품', '디자인',
            '스토리','기승전결','흐름도','결말','서사','이야기','유니버스','전개','시나리오', '개연성', '명료',
            '창의성','창의','신선','독특','참신','발상', '연출','짜임','사실감','구현','현실감','현장감', '활동성','활동력','활동량','움직임','반경',
            '규모','스케일','볼륨','사이즈','크기','넓이','공간감','분량', '공포','공테','무서움','담력','스릴러',
            '문제', '장치', '기계', '센서', '기구', '불친절',
            '메르헨', '커튼콜', '카르텔', '소우주', '풀문', '도고', '플래시', '나우히어', '나비효과', '몽중', '가이드라인',
            '연출력', '짜임새', '공포도', '공포감', '공포심', '약공테', '문제퀄']

# 사전학습된 bert 모델 사용
# num_labels 클래스에 대해서 훈련을 하기 위해서 num_labels=3 할당함, problem_type="multi_label_classification" 를 통해서 모델이 다중 레이블 분류에 해당함을 명시
model = ElectraForSequenceClassification.from_pretrained("monologg/koelectra-small-v3-discriminator", num_labels=3, problem_type="multi_label_classification")
tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator")

# token에 새로운 단어 추가 
tokenizer.add_tokens(addword)
# token에 단어 추가후 기존 모델의 임베딩 레이어에 추가한 단어에 대한 임베딩 벡터가 없을 수 있기 때문
# 아래 코드를 통해서 토큰의 개수가 변했음을 모델에 알리고 모델의 임베딩 레이어를 조정하여 새로운 토큰을 수용할 수 있게 함
model.resize_token_embeddings(len(tokenizer))

In [ ]:
# train content 토큰화
tokenized_train_sentences = tokenizer(
    list(train_data['content']),
    return_tensors="pt",
    max_length=128,
    padding=True,
    truncation=True,
    add_special_tokens=True
)

# test content 토큰화
tokenized_test_sentences = tokenizer(
    list(test_data['content']),
    return_tensors="pt",
    max_length=128,
    padding=True,
    truncation=True,
    add_special_tokens=True
)

# 분류 모델에 넣기 위해서 1차원으로 구성된 세 개의 클래스를 2차원으로 재구성 후 label 로 투입
# -1(부정)=0, 0(중립)=1, 1(긍정)=2

train_label = []
for label in train_data["guide"].values:
    if label == -1:
        train_label.append([1., 0., 0.])
    elif label == 0:
        train_label.append([0., 1., 0.])
    elif label == 1:
        train_label.append([0., 0., 1.])

test_label = []
for label in test_data['guide'].values:
    if label == 0:
        test_label.append([0., 1., 0.])
    elif label == -1:
        test_label.append([1., 0., 0.])
    elif label == 1:
        test_label.append([0., 0., 1.])

# model 에 넣기 위한 dataset 생성 class
class CurseDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item
    
    def __len__(self):
        return len(self.labels)
        
# bert 모델에 데이터가 들어갈 수 있게 만들어 둔 class를 활용하여 train, test dataset 생성
train_dataset = CurseDataset(tokenized_train_sentences, train_label)
test_dataset = CurseDataset(tokenized_test_sentences, test_label)

# hugging face 의 trasformers 라이브러리를 사용하여 모델 훈련시킬 때 사용되는 객체 설정 부분
training_args = TrainingArguments(
    output_dir = './guide_model/check_point/try_1',    # 모델과 훈련 중 생성되는 파일이 저장될 디렉토리 경로
    num_train_epochs = 10,              # 훈련 epoch 수
    per_device_train_batch_size = 16,    # 장치에 할당된 훈련 배치 크기
    per_device_eval_batch_size = 64,    # 장치에 할당된 평가 배치 크기, 모델을 평가할 때 사용되는 배치 크기
    logging_dir = './logs',             # 훈련 중 로그 파일이 저장될 디렉토리
    logging_steps = 500,                # 로그 출력 빈도, 500 step에 한 번씩 출력 예정
    save_total_limit = 2,               # 체크포인트 파일 저장 제한 수
)

# hugging face 의 trasformers 라이브러리를 사용하여 모델 훈련하고 관리하는 객체 설정 부분
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

In [ ]:
# train 진행
trainer.train()

In [ ]:
trainer.save_model("./guide_model/try_1/guide_model1")
tokenizer.save_pretrained("./guide_model/try_1/guide_tokenizer1")

In [4]:
# 저장된 모델과 토크나이저를 불러오기

model_path = "./guide_model/try_1/guide_model1"
tokenizer_path = "./guide_model/try_1/guide_tokenizer1"

model = ElectraForSequenceClassification.from_pretrained(model_path)
tokenizer = ElectraTokenizer.from_pretrained(tokenizer_path)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
# 문장을 입력했을 때, 예측 값을 도출해주는 함수

from torch.nn.functional import softmax

# 함수 정의
def classify_text(text):
    # 문장을 토큰화하고 모델에 입력으로 전달
    text = cleaned_content(text)
    inputs = tokenizer(text, return_tensors="pt")
    outputs = model(**inputs)

    # 소프트맥스 함수를 사용하여 확률값 계산
    probabilities = softmax(outputs.logits, dim=1)

    # 가장 높은 확률값에 해당하는 클래스 선택
    predicted_class = torch.argmax(probabilities).item()

    return predicted_class, probabilities

def change_num(num):
    if num == 0:
        return (-1)
    elif num == 1:
        return 0
    elif num == 2:
        return 1

pred_list = []
for sent in tqdm(test_data['content']):
    pred, _ = classify_text(sent)
    pred_list.append(pred)

test_data['pred'] = pred_list
test_data['pred'] = test_data['pred'].apply(change_num)
test_data
    

  0%|          | 0/1001 [00:00<?, ?it/s]

100%|██████████| 1001/1001 [00:16<00:00, 60.34it/s]


,content,guide,pred
0,#1415_20221128_3인. 흐음.잘 열어보지 않으면,0,0
1,난.바보야.,0,0
2,들어가자마자 커피향이 씨게 낫다 좋앗다 하지만 인테리어는 그냥그랫다. 풀정도 되는듯.,0,0
3,3인. 스토리가 잘 감이 안 잡힌다!,0,0
4,1.5/3인. 활동성 있음.추락조심,0,0
...,...,...,...
999,나에겐 너무 어려웠던. 방탈짬바 있는 분들에게 추천,0,0
1000,2021. 12. 26. (3인) 각자 1인분씩 하고 뿌듯했던 테마.,0,0
1001,첫방 디버프로 초반에 절어버림. 볼것도 못보고 힌트썼는데 잘꾸며놓았다 . 살짝 장치...,0,0
1002,#5 - 2인 - 볼륨에 압도됨 - 히터도 방마다 빵빵함 - 다른 테마도 궁금해짐,0,0


In [6]:
print(train_data['guide'].value_counts())
print(test_data['guide'].value_counts())

guide
 0    3805
-1     162
 1      36
Name: count, dtype: int64
guide
 0    961
-1     35
 1      5
Name: count, dtype: int64


In [7]:
print('0포함', '\n', classification_report(test_data['guide'], test_data['pred']))
test_data2 = test_data[test_data['guide'] != 0]
print('0 제외', '\n', classification_report(test_data2['guide'], test_data2['pred']))

0포함 
               precision    recall  f1-score   support

          -1       0.88      0.86      0.87        35
           0       0.99      1.00      1.00       961
           1       0.00      0.00      0.00         5

    accuracy                           0.99      1001
   macro avg       0.63      0.62      0.62      1001
weighted avg       0.98      0.99      0.99      1001

0 제외 
               precision    recall  f1-score   support

          -1       0.88      0.86      0.87        35
           0       0.00      0.00      0.00         0
           1       0.00      0.00      0.00         5

    accuracy                           0.75        40
   macro avg       0.29      0.29      0.29        40
weighted avg       0.77      0.75      0.76        40



/opt/homebrew/anaconda3/envs/bert_electra/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/anaconda3/envs/bert_electra/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/anaconda3/envs/bert_electra/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier

In [10]:
test = ['가이드 무슨 일?', '가이드 최악', '가이드 엄청 친절하네', 
'스토리 구려', '창의성은 없는데, 인테리어는 좋네요.', '가이드 좋아용']

for sent in test:
    pred, _ = classify_text(sent)
    print(f'문장: {sent}, 평가: {pred}')
    print("-"*10, '\n')

문장: 가이드 무슨 일?, 평가: 0
---------- 

문장: 가이드 최악, 평가: 0
---------- 

문장: 가이드 엄청 친절하네, 평가: 0
---------- 

문장: 스토리 구려, 평가: 1
---------- 

문장: 창의성은 없는데, 인테리어는 좋네요., 평가: 1
---------- 

문장: 가이드 좋아용, 평가: 0
---------- 



## 1차 결과
- 부정 결과가 많으니까 부정으로 편향된 결과!

# 2차 모델

In [4]:
# labeling 된 데이터 불러오기
survey = pd.read_csv("./_data/survey.csv", index_col=0)
survey.fillna(0, inplace=True)
survey = survey.reset_index()

# 데이터 중에서 작업을 하고자 하는 라벨만 가져오기
guide_data = survey[['content_id', 'guide']]

# content_id 중에서 댓글이 같이 추가된 데이터 정리
guide_data['content_id'] = guide_data['content_id'].apply(lambda x: x.split(',')[0] if len(str(x)) > 6 else x)

# content_id 가 0인 데이터 제외 -> content_id 모두 int type 으로 변경 (추후 Merge 를 위함)
guide_data = guide_data[guide_data['content_id'] != 0].reset_index().drop(columns=['index'])
guide_data['content_id'] = guide_data['content_id'].astype(int)

# 전체 review data 불러오기
review_all = pd.read_csv("./_data/reviews.csv", index_col=0)

# 전체 review data 중에서 survey data에 있는 댓글만 가져오기
survey_content = review_all.loc[review_all['id'].isin(guide_data['content_id'])][['id', 'content']]
# merge 를 위해서 id 컬럼명 통일하기
survey_content.columns=['content_id', 'content']

# 통일된 content_id 를 기반으로 데이터 merge
guide_data = pd.merge(guide_data, survey_content)

# 추후 원활한 계산을 위해서 숫자 부분은 모두 int 로 바꿔줌
guide_data['guide'] = guide_data['guide'].astype(int)

# 최소한의 전처리
def cleaned_content(text):
    d = re.sub('\n', '. ', text) # 줄바꿈 > .
    d = re.sub('[^가-힣0-9a-zA-Z ]{2,}', ".", d) # 특수문자 두개 이상인거 .으로 변경
    return d

guide_data['content'] = guide_data['content'].apply(cleaned_content)

final_df = guide_data[['content', 'guide']]

from kiwipiepy import Kiwi
kiwi = Kiwi()

def kiwi_clean(text):
    get_kiwi_pos = ['NNG', 'NP', 'NNP', 'MM', 'VV', 'VV-I', 'VV-R', 'VA', 'VA-I', 'VA-R', 'VCP', 'VCN', 'MAG', 'MAJ', 'XR']
    kiwi_lem = []
    for word in kiwi.tokenize(text):
        if word.tag in get_kiwi_pos:
            kiwi_lem.append(word.lemma)
    return ' '.join(kiwi_lem)

final_df['content'] = final_df['content'].apply(kiwi_clean)

# 라벨별 개수 확인
print(final_df['guide'].value_counts())
print('\n')
train_data = final_df.sample(frac=0.8, random_state=42).reset_index().drop(columns='index')
print('train_data', train_data['guide'].value_counts())
test_data = final_df.drop(train_data.index).reset_index().drop(columns='index')
print('\n', 'test_data', test_data['guide'].value_counts())
print('\n')
# 중복 데이터 제거(데이터 분리 후 중복이 생길 수 있어서 데이터 분리 후 중복 데이터 처리 진행)

# 데이터셋 개수 확인
print('중복 제거 전 학습 데이터셋: {}'.format(len(train_data)))
print('중복 제거 전 테스트 데이터셋: {}'.format(len(test_data)))
print('\n')
# 중복 데이터 제거
train_data.drop_duplicates(subset=['content'], inplace=True)
test_data.drop_duplicates(subset=['content'], inplace=True)
print('\n')
# 데이터셋 개수 확인
print('중복 제거 후 학습 데이터셋: {}'.format(len(train_data)))
print('중복 제거 후 테스트 데이터셋: {}'.format(len(test_data)))

/var/folders/t4/314qsssd1p718wry9j0bh9000000gn/T/ipykernel_68959/144630509.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  guide_data['content_id'] = guide_data['content_id'].apply(lambda x: x.split(',')[0] if len(str(x)) > 6 else x)


guide
 0    4774
-1     200
 1      49
Name: count, dtype: int64


train_data guide
 0    3820
-1     162
 1      36
Name: count, dtype: int64

 test_data guide
 0    965
-1     35
 1      5
Name: count, dtype: int64


중복 제거 전 학습 데이터셋: 4018
중복 제거 전 테스트 데이터셋: 1005




중복 제거 후 학습 데이터셋: 3919
중복 제거 후 테스트 데이터셋: 989


/var/folders/t4/314qsssd1p718wry9j0bh9000000gn/T/ipykernel_68959/144630509.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['content'] = final_df['content'].apply(kiwi_clean)


In [3]:
# 토큰에 추가할 단어 -> '방탈출'이라는 도메인 지시기에 근거한 용어, 분리되어서는 안 되기 때문에 별도로 추가 작업 진행
addword = ['공테', '약공테', '감테', '창공', '갑툭튀', '삐딱', '삑', '꽝', '삑딱쾅', '삑딱쾅', '삑딱', '쫄', '극쫄',
            '극극쫄', '쫄팟', '쫄탱', '쫄보', '극', '약탱', '탱쫄', '극극극', '뉴비', '하드캐리', '극혐', '피지컬',
            '어거지', '뚝배기', '뚝문', '셀뚝', '억까', '트롤링', '트롤짓', '흙길', '풀길', '꽃길', '풀꽃', '꽃다발',
            '꽃밭', '웰메이드', '인생테마', '머글', '방린이', '방유아', '방세포', '방태아', '방탈러', '과몰입러', '옵저버',
            '리트', '연방', '혼방', '혼불', '워킹', '워크인', '장치방', '문제방', '직렬', '병렬', '육각형', '볼드', '볼드충', '에바',
            '가이드', '조도', '조명', '밝기', '어두움', '인테리어', '비주얼', '소품', '디자인',
            '스토리','기승전결','흐름도','결말','서사','이야기','유니버스','전개','시나리오', '개연성', '명료',
            '창의성','창의','신선','독특','참신','발상', '연출','짜임','사실감','구현','현실감','현장감', '활동성','활동력','활동량','움직임','반경',
            '규모','스케일','볼륨','사이즈','크기','넓이','공간감','분량', '공포','공테','무서움','담력','스릴러',
            '문제', '장치', '기계', '센서', '기구', '불친절',
            '메르헨', '커튼콜', '카르텔', '소우주', '풀문', '도고', '플래시', '나우히어', '나비효과', '몽중', '가이드라인',
            '연출력', '짜임새', '공포도', '공포감', '공포심', '약공테', '문제퀄']

# 사전학습된 bert 모델 사용
# num_labels 클래스에 대해서 훈련을 하기 위해서 num_labels=3 할당함, problem_type="multi_label_classification" 를 통해서 모델이 다중 레이블 분류에 해당함을 명시
model = ElectraForSequenceClassification.from_pretrained("monologg/koelectra-small-v3-discriminator", num_labels=3, problem_type="multi_label_classification")
tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator")

# token에 새로운 단어 추가 
tokenizer.add_tokens(addword)
# token에 단어 추가후 기존 모델의 임베딩 레이어에 추가한 단어에 대한 임베딩 벡터가 없을 수 있기 때문
# 아래 코드를 통해서 토큰의 개수가 변했음을 모델에 알리고 모델의 임베딩 레이어를 조정하여 새로운 토큰을 수용할 수 있게 함
model.resize_token_embeddings(len(tokenizer))

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-small-v3-discriminator and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Embedding(35080, 128)

In [4]:
# train content 토큰화
tokenized_train_sentences = tokenizer(
    list(train_data['content']),
    return_tensors="pt",
    max_length=128,
    padding=True,
    truncation=True,
    add_special_tokens=True
)

# test content 토큰화
tokenized_test_sentences = tokenizer(
    list(test_data['content']),
    return_tensors="pt",
    max_length=128,
    padding=True,
    truncation=True,
    add_special_tokens=True
)

# 분류 모델에 넣기 위해서 1차원으로 구성된 세 개의 클래스를 2차원으로 재구성 후 label 로 투입
# -1(부정)=0, 0(중립)=1, 1(긍정)=2

train_label = []
for label in train_data["guide"].values:
    if label == -1:
        train_label.append([1., 0., 0.])
    elif label == 0:
        train_label.append([0., 1., 0.])
    elif label == 1:
        train_label.append([0., 0., 1.])

test_label = []
for label in test_data['guide'].values:
    if label == 0:
        test_label.append([0., 1., 0.])
    elif label == -1:
        test_label.append([1., 0., 0.])
    elif label == 1:
        test_label.append([0., 0., 1.])

# model 에 넣기 위한 dataset 생성 class
class CurseDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item
    
    def __len__(self):
        return len(self.labels)
        
# bert 모델에 데이터가 들어갈 수 있게 만들어 둔 class를 활용하여 train, test dataset 생성
train_dataset = CurseDataset(tokenized_train_sentences, train_label)
test_dataset = CurseDataset(tokenized_test_sentences, test_label)

# hugging face 의 trasformers 라이브러리를 사용하여 모델 훈련시킬 때 사용되는 객체 설정 부분
training_args = TrainingArguments(
    output_dir = './guide_model/check_point/try_2',    # 모델과 훈련 중 생성되는 파일이 저장될 디렉토리 경로
    num_train_epochs = 30,              # 훈련 epoch 수
    per_device_train_batch_size = 16,    # 장치에 할당된 훈련 배치 크기
    per_device_eval_batch_size = 64,    # 장치에 할당된 평가 배치 크기, 모델을 평가할 때 사용되는 배치 크기
    logging_dir = './logs',             # 훈련 중 로그 파일이 저장될 디렉토리
    logging_steps = 500,                # 로그 출력 빈도, 500 step에 한 번씩 출력 예정
    save_total_limit = 2,               # 체크포인트 파일 저장 제한 수
)

# hugging face 의 trasformers 라이브러리를 사용하여 모델 훈련하고 관리하는 객체 설정 부분
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

In [5]:
# train 진행
trainer.train()

  0%|          | 0/7350 [00:00<?, ?it/s]/var/folders/t4/314qsssd1p718wry9j0bh9000000gn/T/ipykernel_67232/3043833792.py:49: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
  7%|▋         | 500/7350 [02:02<27:31,  4.15it/s]

{'loss': 0.2139, 'learning_rate': 4.6598639455782315e-05, 'epoch': 2.04}


/var/folders/t4/314qsssd1p718wry9j0bh9000000gn/T/ipykernel_67232/3043833792.py:49: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
 14%|█▎        | 1000/7350 [04:02<25:12,  4.20it/s]

{'loss': 0.1273, 'learning_rate': 4.319727891156463e-05, 'epoch': 4.08}


/var/folders/t4/314qsssd1p718wry9j0bh9000000gn/T/ipykernel_67232/3043833792.py:49: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
 20%|██        | 1500/7350 [06:03<23:25,  4.16it/s]

...

/var/folders/t4/314qsssd1p718wry9j0bh9000000gn/T/ipykernel_67232/3043833792.py:49: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
 95%|█████████▌| 7000/7350 [23:57<00:59,  5.88it/s]

{'loss': 0.0074, 'learning_rate': 2.3809523809523808e-06, 'epoch': 28.57}


/var/folders/t4/314qsssd1p718wry9j0bh9000000gn/T/ipykernel_67232/3043833792.py:49: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
100%|██████████| 7350/7350 [24:58<00:00,  4.91it/s]

{'train_runtime': 1498.1406, 'train_samples_per_second': 78.477, 'train_steps_per_second': 4.906, 'train_loss': 0.053807958129311904, 'epoch': 30.0}


TrainOutput(global_step=7350, training_loss=0.053807958129311904, metrics={'train_runtime': 1498.1406, 'train_samples_per_second': 78.477, 'train_steps_per_second': 4.906, 'train_loss': 0.053807958129311904, 'epoch': 30.0})

In [6]:
trainer.save_model("./guide_model/try_2/guide_model2")
tokenizer.save_pretrained("./guide_model/try_2/guide_tokenizer2")

('./guide_model/try_2/guide_tokenizer2/tokenizer_config.json',
 './guide_model/try_2/guide_tokenizer2/special_tokens_map.json',
 './guide_model/try_2/guide_tokenizer2/vocab.txt',
 './guide_model/try_2/guide_tokenizer2/added_tokens.json')

In [2]:
# 저장된 모델과 토크나이저를 불러오기

model_path = "./guide_model/try_2/guide_model2"
tokenizer_path = "./guide_model/try_2/guide_tokenizer2"

model = ElectraForSequenceClassification.from_pretrained(model_path)
tokenizer = ElectraTokenizer.from_pretrained(tokenizer_path)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
# 문장을 입력했을 때, 예측 값을 도출해주는 함수

from torch.nn.functional import softmax

# 함수 정의
def classify_text(text):
    # 문장을 토큰화하고 모델에 입력으로 전달
    text = cleaned_content(text)
    text = kiwi_clean(text)
    inputs = tokenizer(text, return_tensors="pt")
    outputs = model(**inputs)

    # 소프트맥스 함수를 사용하여 확률값 계산
    probabilities = softmax(outputs.logits, dim=1)

    # 가장 높은 확률값에 해당하는 클래스 선택
    predicted_class = torch.argmax(probabilities).item()

    return predicted_class, probabilities

def change_num(num):
    if num == 0:
        return (-1)
    elif num == 1:
        return 0
    elif num == 2:
        return 1

pred_list = []
for sent in tqdm(test_data['content']):
    pred, _ = classify_text(sent)
    pred_list.append(pred)

test_data['pred'] = pred_list
test_data['pred'] = test_data['pred'].apply(change_num)

print('0포함', '\n', classification_report(test_data['guide'], test_data['pred']))
test_data2 = test_data[test_data['guide'] != 0]
print('0 제외', '\n', classification_report(test_data2['guide'], test_data2['pred']))

test = ['가이드 무슨 일?', '가이드 최악', '가이드 엄청 친절하네', 
'스토리 구려', '창의성은 없는데, 인테리어는 좋네요.', '가이드 좋아용']

for sent in test:
    pred, _ = classify_text(sent)
    print(f'문장: {sent}, 평가: {pred}')
    print("-"*10, '\n')

100%|██████████| 989/989 [00:10<00:00, 90.76it/s]
/opt/homebrew/anaconda3/envs/bert_electra/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/anaconda3/envs/bert_electra/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/anaconda3/envs/bert_electra/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _

0포함 
               precision    recall  f1-score   support

          -1       0.94      0.91      0.93        35
           0       1.00      1.00      1.00       949
           1       0.67      0.80      0.73         5

    accuracy                           0.99       989
   macro avg       0.87      0.90      0.88       989
weighted avg       0.99      0.99      0.99       989

0 제외 
               precision    recall  f1-score   support

          -1       0.97      0.91      0.94        35
           0       0.00      0.00      0.00         0
           1       0.80      0.80      0.80         5

    accuracy                           0.90        40
   macro avg       0.59      0.57      0.58        40
weighted avg       0.95      0.90      0.92        40

문장: 가이드 무슨 일?, 평가: 0
---------- 

문장: 가이드 최악, 평가: 2
---------- 

문장: 가이드 엄청 친절하네, 평가: 2
---------- 

문장: 스토리 구려, 평가: 1
---------- 

문장: 창의성은 없는데, 인테리어는 좋네요., 평가: 1
---------- 

문장: 가이드 좋아용, 평가: 1
---------- 

